In [1]:
from config import api_key
from yelpapi import YelpAPI
import requests
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

In [99]:
cities = ["Anaheim,CA", "Santa Ana,CA", "Irvine,CA", "Huntington Beach,CA", "Garden Grove,CA", "Orange,CA", "Fullerton,CA", "Costa Mesa,CA", "Mission Viejo,CA","Westminster,CA"]
url = "https://api.yelp.com/v3/businesses/search"
headers = {'Authorization': 'Bearer %s' %api_key}
price_locations = {}

for city in cities:
    locations={}
    params = {"term":"restaurant",
              "location":city,
              "limit":50,
             "radius":5000}
    responses=requests.get(url,headers = headers, params = params).json()
    for response in responses["businesses"]:
        try:
            money= response["price"]
            if money not in locations:
                locations[money]=1
                
            else:
                locations[money]+=1
        except KeyError:
            continue  
    price_locations[city]=locations 
    url_params = {"term":"restaurant",
                  "location":city,
                  "limit":50,
                 "offset":50,
                  "radius":5000}
    results = requests.get(url,headers = headers, params = url_params).json()
    for r in results["businesses"]:
        try:
            m = r["price"]
            if m not in locations:
                locations[m]=1
            else:
                locations[m]+=1
        except KeyError:
            continue
    price_locations[city]=locations
print(price_locations)        
     

{'Anaheim,CA': {'$$': 62, '$$$': 8, '$': 26, '$$$$': 3}, 'Santa Ana,CA': {'$$': 47, '$$$$': 1, '$': 49, '$$$': 1}, 'Irvine,CA': {'$$$': 2, '$$': 63, '$': 34}, 'Huntington Beach,CA': {'$$': 72, '$': 22, '$$$': 4}, 'Garden Grove,CA': {'$$': 48, '$': 49, '$$$$': 2}, 'Orange,CA': {'$$': 50, '$': 46, '$$$': 1, '$$$$': 2}, 'Fullerton,CA': {'$$': 59, '$': 35, '$$$': 3}, 'Costa Mesa,CA': {'$$': 68, '$': 26, '$$$': 4, '$$$$': 2}, 'Mission Viejo,CA': {'$': 40, '$$': 57, '$$$': 2}, 'Westminster,CA': {'$$': 39, '$': 60}}


In [102]:
columns = ["Cheap","Affordable","Expensive","Luxury"]
DF=pd.DataFrame.from_dict(price_locations).T #.T means transpose, switch columns and rows
DF = DF.reset_index()
# df.rename(columns = {'index':'City','$':'Cheap'}, inplace = True)
df = DF.rename(columns={"$":"Cheap","$$":"Affordable","$$$":"Expensive","$$$$":"Luxury","index":"City"})
# DF.to_csv("output.csv")

df = df.fillna(0)
df

,City,Cheap,Affordable,Expensive,Luxury
0,"Anaheim,CA",26.0,62.0,8.0,3.0
1,"Costa Mesa,CA",26.0,68.0,4.0,2.0
2,"Fullerton,CA",35.0,59.0,3.0,0.0
3,"Garden Grove,CA",49.0,48.0,0.0,2.0
4,"Huntington Beach,CA",22.0,72.0,4.0,0.0
5,"Irvine,CA",34.0,63.0,2.0,0.0
6,"Mission Viejo,CA",40.0,57.0,2.0,0.0
7,"Orange,CA",46.0,50.0,1.0,2.0
8,"Santa Ana,CA",49.0,47.0,1.0,1.0
9,"Westminster,CA",60.0,39.0,0.0,0.0
